# Object Proposals

## Selective Search

https://github.com/nipunmanral/Object-Detection-using-OpenCV/blob/master/selective_search.py

In [1]:
import cv2
import os

In [2]:
im_dir = "/home/crdw/suraj/data/dishwasher/CocoFormat/coco_dishwasher_im_data/data"
img = cv2.imread(os.path.join(im_dir, "uneven_alt_2_im_green_bg__2021-08-0518:43:54.018043.png"))

In [49]:
def compute_selective_search_proposals(img, strategy):
    # Instantiate selective search
    ss = cv2.ximgproc.segmentation.createSelectiveSearchSegmentation()

    # Convert image from BGR to RGB
    rgb_im = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    ss.addImage(rgb_im)
    gs = cv2.ximgproc.segmentation.createGraphSegmentation()

    ss.addGraphSegmentation(gs)

    # Create strategy using color similarity
    if strategy == "color":
        strategy_color = cv2.ximgproc.segmentation.createSelectiveSearchSegmentationStrategyColor()
        ss.addStrategy(strategy_color)
    elif strategy == "all":
        # Create strategy using all similarities (size, color, fill, texture)
        strategy_color = cv2.ximgproc.segmentation.createSelectiveSearchSegmentationStrategyColor()
        strategy_fill = cv2.ximgproc.segmentation.createSelectiveSearchSegmentationStrategyFill()
        strategy_size = cv2.ximgproc.segmentation.createSelectiveSearchSegmentationStrategySize()
        strategy_texture = cv2.ximgproc.segmentation.createSelectiveSearchSegmentationStrategyTexture()
        strategy_multiple = cv2.ximgproc.segmentation.createSelectiveSearchSegmentationStrategyMultiple(
            strategy_color, strategy_fill, strategy_size, strategy_texture)
        ss.addStrategy(strategy_multiple)

    return ss.process()

In [8]:
print("Total proposals = ", len(get_boxes))

Total proposals =  269


In [48]:
# Set limit to number of proposed boxes
proposal_box_limit = 100

In [12]:
# Convert (x,y,w,h) parameters for the top 100 proposal boxes returned from ss.process() command into
# (x, y, x+w, y+h) parameters to be consistent with the xml tags of the ground truth boxes where
# (x,y) indicates the top left corner and (x+w,y+h) indicates the bottom right corner of bounding box

boxes = [[box[0], box[1], box[0] + box[2], box[1] + box[3]] for box in get_boxes[0:proposal_box_limit]]

In [13]:
boxes

[[323, 594, 404, 596],
 [237, 89, 280, 116],
 [444, 38, 627, 49],
 [194, 250, 277, 304],
 [208, 121, 225, 143],
 [639, 158, 670, 208],
 [132, 583, 282, 591],
 [650, 483, 670, 502],
 [429, 591, 554, 594],
 [307, 123, 342, 154],
 [258, 119, 301, 202],
 [223, 106, 253, 124],
 [12, 571, 20, 609],
 [51, 60, 414, 75],
 [210, 594, 323, 596],
 [623, 469, 630, 547],
 [598, 566, 617, 593],
 [135, 111, 189, 155],
 [597, 556, 628, 579],
 [409, 616, 647, 620],
 [325, 262, 344, 284],
 [270, 47, 404, 49],
 [510, 589, 606, 592],
 [9, 371, 11, 541],
 [614, 268, 625, 361],
 [13, 47, 371, 61],
 [15, 562, 60, 589],
 [371, 44, 490, 47],
 [228, 27, 348, 33],
 [640, 222, 666, 481],
 [643, 347, 646, 429],
 [323, 45, 383, 48],
 [261, 584, 478, 590],
 [563, 599, 633, 604],
 [20, 423, 54, 569],
 [10, 195, 12, 383],
 [118, 257, 287, 324],
 [626, 503, 640, 562],
 [240, 598, 606, 605],
 [214, 250, 230, 265],
 [638, 113, 661, 190],
 [386, 587, 517, 590],
 [19, 572, 52, 609],
 [159, 219, 190, 261],
 [0, 56, 14, 620],

In [12]:
output_img_proposal_top100 = img.copy()

In [13]:
output_img_iou_qualified = img.copy()

In [14]:
output_img_final = img.copy()

In [15]:
for i in range(0, len(boxes)):
    top_x, top_y, bottom_x, bottom_y = boxes[i]
    cv2.rectangle(output_img_proposal_top100, (top_x, top_y), (bottom_x, bottom_y), (0, 255, 0), 1, cv2.LINE_AA)

In [17]:
cv2.imwrite("selective_search_output.png", output_img_proposal_top100)

True

## Edge Boxes
https://docs.opencv.org/3.4/d4/d0d/group__ximgproc__edgeboxes.html

In [1]:
import cv2
import os
import numpy as np
import json
import os

In [2]:
im_dir = "/home/crdw/suraj/data/dishwasher/CocoFormat/coco_dishwasher_im_data/data"
img = cv2.imread(os.path.join(im_dir, "uneven_alt_2_im_green_bg__2021-08-0518:43:54.018043.png"))

In [3]:
model_path = "proposal/model.yml.gz"

In [4]:
edge_detection_obj = cv2.ximgproc.createStructuredEdgeDetection(model_path)

In [5]:
def compute_edge_boxes(img):
    rgb_im = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Get the edges
    edges = edge_detection_obj.detectEdges(np.float32(rgb_im)/255.0)

    # Create an orientation map
    orient_map = edge_detection_obj.computeOrientation(edges)

    # Suppress edges
    edges = edge_detection_obj.edgesNms(edges, orient_map)

    # Create edge box:
    edge_boxes = cv2.ximgproc.createEdgeBoxes()

    edge_boxes.setMaxBoxes(100)
    edge_boxes.setAlpha(0.5)
    edge_boxes.setBeta(0.5)
    prop_boxes, scores = edge_boxes.getBoundingBoxes(edges, orient_map)
    
    return prop_boxes, scores

In [6]:
box, score = compute_edge_boxes(img)

In [7]:
box

array([[202,  93, 101, 135],
       [187,  90, 181, 239],
       [ 19, 555,  41,  17],
       [144, 101, 160, 230],
       [261, 118, 111, 186],
       [148, 216, 147, 111],
       [174,  39, 491, 572],
       [  6,  75, 365, 253],
       [596, 550,  55,  45],
       [ 11, 565,  51,  47],
       [  7,  82, 363, 529],
       [465,  90, 201, 519],
       [259, 125,  47,  86],
       [187,   4, 473, 324],
       [ 19, 543,  51,  29],
       [135,  90, 230, 128],
       [629, 556,  25,  57],
       [ 17, 466, 637, 143],
       [242, 228,  53,  69],
       [ 16,  12,  76,  45],
       [235, 256,  45,  41],
       [581, 520,  78,  89],
       [ 11, 567,  21,  44],
       [197,  22, 193, 207],
       [476, 513, 155,  87],
       [187, 242, 136,  85],
       [ 57, 186, 238, 155],
       [219, 207, 100,  90],
       [173,  90, 240, 519],
       [190, 183, 171, 115],
       [  8, 169, 190, 444],
       [ 19,  40,  41,  17],
       [ 19,  51,  41,  17],
       [458, 401, 199, 209],
       [ 94, 5

In [8]:
score

array([[0.16737314],
       [0.16250858],
       [0.14363495],
       [0.13923028],
       [0.13448931],
       [0.12768506],
       [0.11523562],
       [0.11407924],
       [0.11289115],
       [0.1126641 ],
       [0.10216863],
       [0.09929614],
       [0.09299069],
       [0.09290861],
       [0.09263022],
       [0.09235998],
       [0.08617897],
       [0.08391323],
       [0.08318558],
       [0.08269278],
       [0.07790691],
       [0.07680444],
       [0.07556385],
       [0.07453776],
       [0.07440846],
       [0.07432207],
       [0.07110319],
       [0.07103984],
       [0.06982406],
       [0.06856178],
       [0.06844934],
       [0.06697955],
       [0.06697955],
       [0.06694882],
       [0.06674829],
       [0.06670367],
       [0.06629159],
       [0.06618499],
       [0.06601085],
       [0.06546346],
       [0.06473204],
       [0.0641657 ],
       [0.06372797],
       [0.06268795],
       [0.06049023],
       [0.06009348],
       [0.05991514],
       [0.058

In [9]:
score = score + 0.5

In [10]:
score

array([[0.6673731 ],
       [0.6625086 ],
       [0.6436349 ],
       [0.63923025],
       [0.6344893 ],
       [0.62768507],
       [0.6152356 ],
       [0.61407924],
       [0.61289114],
       [0.6126641 ],
       [0.6021686 ],
       [0.59929615],
       [0.5929907 ],
       [0.5929086 ],
       [0.5926302 ],
       [0.59235996],
       [0.58617896],
       [0.5839132 ],
       [0.58318555],
       [0.5826928 ],
       [0.5779069 ],
       [0.57680446],
       [0.57556385],
       [0.57453775],
       [0.5744085 ],
       [0.57432204],
       [0.5711032 ],
       [0.57103986],
       [0.56982404],
       [0.5685618 ],
       [0.5684493 ],
       [0.5669795 ],
       [0.5669795 ],
       [0.56694883],
       [0.56674826],
       [0.5667037 ],
       [0.5662916 ],
       [0.566185  ],
       [0.56601083],
       [0.5654635 ],
       [0.564732  ],
       [0.5641657 ],
       [0.563728  ],
       [0.56268793],
       [0.56049025],
       [0.56009346],
       [0.5599151 ],
       [0.558

In [8]:
input_dir = "/home/crdw/suraj/data/dishwasher/CocoFormat/test_data/"

In [9]:
with open(os.path.join(input_dir, "labels.json")) as f:
    ann_data = json.load(f)

In [13]:
ann_data['images'][0]

{'id': 1,
 'file_name': 'bowl_2021-09-0718:52:04.973487_white.png',
 'height': 620,
 'width': 670,
 'license': None,
 'coco_url': None}

In [14]:
ann_data['images'][0]['file_name']

'bowl_2021-09-0718:52:04.973487_white.png'

In [15]:
len(ann_data['images'])

50

In [38]:
boxes = []
scores = []
indexes = []

im_dir = "data"

for i in range(len(ann_data['images'])):
    # print(ann_data['images'][i]['file_name'])
    # print(ann_data['images'][i]['id'])
    # Image id
    index = ann_data['images'][i]['id']
    # Image name
    file_name = ann_data['images'][i]['file_name']
    # Read image
    img = cv2.imread(os.path.join(input_dir, im_dir, file_name))
    
    # Obtain bounding boxes and scores
    box, score = compute_edge_boxes(img)
    
    # Convert to dtype float32
    box = box.astype(np.float32)
    boxes.append(box)
    scores.append(score)
    indexes.append(index)

In [39]:
proposal = {
    'boxes': boxes,
    'scores': scores,
    'indexes': indexes,
}

In [40]:
proposal['boxes']

[array([[ 59., 461.,  79.,  97.],
        [ 27., 388., 195., 195.],
        [ 16.,  58., 293., 525.],
        [ 16.,  39., 652., 572.],
        [ 49., 432., 126., 136.],
        [ 85., 463.,  53.,  68.],
        [ 14., 391., 640., 217.],
        [482., 466., 173., 130.],
        [160., 510.,  51.,  29.],
        [618., 496.,  35.,  67.],
        [ 22., 185., 200., 373.],
        [117., 441., 100., 110.],
        [166., 487.,  51.,  31.],
        [107., 488.,  17.,  40.],
        [125., 445.,  47., 106.],
        [599., 552.,  52.,  47.],
        [598., 476.,  55., 107.],
        [620., 585.,  34.,  31.],
        [160., 465.,  57.,  33.],
        [552., 105., 115., 398.],
        [ 58., 461.,  31.,  77.],
        [162., 463.,  53.,  74.],
        [498., 529., 130.,  67.],
        [ 51., 428., 106.,  49.],
        [ 22., 561.,  37.,  37.],
        [ 32., 390., 182.,  91.],
        [150., 429.,  66.,  55.],
        [ 73., 534.,  66.,  41.],
        [153., 425.,  72., 111.],
        [ 94.,

In [41]:
proposal['scores']

[array([[0.18915045],
        [0.1663215 ],
        [0.1448743 ],
        [0.1443283 ],
        [0.13184209],
        [0.12579842],
        [0.11043292],
        [0.09485849],
        [0.0909081 ],
        [0.08936132],
        [0.08731747],
        [0.08680894],
        [0.08583546],
        [0.08420888],
        [0.08076762],
        [0.07954091],
        [0.07795347],
        [0.0771402 ],
        [0.07712968],
        [0.07661922],
        [0.07610882],
        [0.07408386],
        [0.0703389 ],
        [0.06974251],
        [0.06812762],
        [0.06801631],
        [0.06606442],
        [0.06539533],
        [0.06458274],
        [0.06351136],
        [0.06259245],
        [0.06120483],
        [0.06064944],
        [0.05842756],
        [0.05806881],
        [0.05702008],
        [0.05548473],
        [0.05462164],
        [0.05418538],
        [0.05411406],
        [0.05301197],
        [0.05272956],
        [0.05182445],
        [0.05180046],
        [0.05139879],
        [0

In [42]:
proposal['indexes']

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50]

In [43]:
import pickle

In [44]:
with open('edge_boxes_dishwasher_test_images.pkl', 'wb') as handle:
    pickle.dump(proposal, handle)

In [45]:
with open('edge_boxes_dishwasher_test_images.pkl', 'rb') as f:
    loaded_proposals = pickle.load(f, encoding="latin1")